# Machine Learning Model
First we need to install PyTorch.

In [1]:
!pip install torch torchvision

## Imports

In [2]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


## Load Data Set

In [4]:
class RainDataset(Dataset):
    def __init__(self, data_type: str):
        combined_with_perc: pd.DataFrame = pd.read_hdf("combined_with_perc.hdf5", data_type)
        x = combined_with_perc[['det_run', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10',
       'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20',
       'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29', 'E30',
       'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E40',
       'E41', 'E42', 'E43', 'E44', 'E45', 'E46', 'E47', 'E48', 'E49', 'E50',
       'E51', 'is-260', 'is-310', 'is-240', 'DD', 'FH', 'T', 'P', 'N',
       'U']].values
        y = combined_with_perc[["RH-fix"]].values

        self.x_train=torch.tensor(x,dtype=torch.float32)
        self.y_train=torch.tensor(y,dtype=torch.float32)

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, idx):
        return self.x_train[idx], self.y_train[idx]

In [5]:
model_dataset = RainDataset("model")
test_dataset = RainDataset("test")

In [6]:
train_dataloader = DataLoader(model_dataset, batch_size=32)
test_dataloader = DataLoader(test_dataset, batch_size=32)

## Create model

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(61, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        params = self.linear_relu_stack(x)
        return params

model = NeuralNetwork()

## Loss function

In [8]:
learning_rate = 1e-4
batch_size = 32
epochs = 20
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Run model

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)

        loss = loss_fn(pred, y)

        # Backpropagation
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    # test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 104.988441  [    0/ 4109]
loss: 31.996218  [ 3200/ 4109]
Epoch 2
-------------------------------
loss: 20.106495  [    0/ 4109]
loss: 28.511253  [ 3200/ 4109]
Epoch 3
-------------------------------
loss: 16.791094  [    0/ 4109]
loss: 27.102640  [ 3200/ 4109]
Epoch 4
-------------------------------
loss: 15.138383  [    0/ 4109]
loss: 27.953651  [ 3200/ 4109]
Epoch 5
-------------------------------
loss: 14.586462  [    0/ 4109]
loss: 28.404106  [ 3200/ 4109]
Epoch 6
-------------------------------
loss: 14.348567  [    0/ 4109]
loss: 27.596643  [ 3200/ 4109]
Epoch 7
-------------------------------
loss: 14.356636  [    0/ 4109]
loss: 27.407780  [ 3200/ 4109]
Epoch 8
-------------------------------
loss: 14.356996  [    0/ 4109]
loss: 27.144247  [ 3200/ 4109]
Epoch 9
-------------------------------
loss: 14.379161  [    0/ 4109]
loss: 27.022127  [ 3200/ 4109]
Epoch 10
-------------------------------
loss: 14.419434  [    0/ 4109]
loss: 26.

In [11]:
combined_with_perc = pd.read_hdf("combined_with_perc.hdf5", "test")
combined_with_perc

,,det_run,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,median_pred_D-NE,median_pred_D-E,median_pred_D-SE,median_pred_D-S,median_pred_D-SW,median_pred_D-W,median_pred_D-NW,num_pos,num_high_pos,num_very_pos
date,loc_nr,,,,,,,,,,,,,,,,,,,,,
2020-01-01,260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2020-01-02,260,2.0,3.0,3.0,3.0,2.0,4.0,2.0,2.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.5,0.0,50,0,0
2020-01-03,260,50.0,37.0,49.0,33.0,52.0,28.0,33.0,38.0,47.0,31.0,...,0.0,0.0,0.0,0.0,0.0,42.0,0.0,50,50,0
2020-01-04,260,1.0,1.0,1.0,5.0,3.0,0.0,1.0,0.0,5.0,3.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,45,0,0
2020-01-05,260,0.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,48,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-26,240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2021-04-27,240,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,0,0
2021-04-28,240,41.0,27.0,10.0,38.0,29.0,5.0,31.0,14.0,32.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,21.0,50,25,0


In [12]:
x = torch.tensor(combined_with_perc[['det_run', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10',
       'E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E18', 'E19', 'E20',
       'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29', 'E30',
       'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E40',
       'E41', 'E42', 'E43', 'E44', 'E45', 'E46', 'E47', 'E48', 'E49', 'E50',
       'E51', 'is-260', 'is-310', 'is-240', 'DD', 'FH', 'T', 'P', 'N',
       'U']].values, dtype=torch.float32)

model.eval()
y_hat = model(x).detach().numpy().transpose()[0]
y = combined_with_perc["RH-fix"].values

print(f"RMSE: {np.sqrt(((y - y_hat) ** 2).mean())}")
print(f"MAE: {np.abs(y - y_hat).mean()}")
# print(f"SMAE: {np.abs(y - y_hat).mean()}")

RMSE: 28.348249570600718
MAE: 11.24245226487671
